In [1]:
import pandas as pd
import numpy as np

# 1. PDBbind v2020

In [2]:
cluster_result=pd.read_csv("/pubhome/hzhu02/models/Redocked_pose/split_dataset/cluster/pdbbind_2020_cluster_result.csv")

In [3]:
cluster_result.columns

Index(['pdb', 'affinity', 'PCV_cluster', 'main_chain', 'res_num', 'pfam',
       'pfam_name', 'pdb_start', 'pdb_end', 'cov_num', 'clan_acc', 'clan_id',
       'uniport', 'seq', 'mol_weight', 'SCV_cluster', 'smina', 'year', 'Type',
       'heavy_atom_num', 'ligand_sasa', 'protein_sasa', 'complex_sasa',
       'delta_sasa'],
      dtype='object')

## criteria 1: resolution < 3A ---> 18274

In [10]:
# if method was NMR, it was annotated as 100
resolution={}
with open ("/pubhome/hzhu02/GPSF/dataset/pdbbind_v2020/index/INDEX_general_PL.2020", 'r') as f:
    for line in f.readlines():
        if not line.startswith("#"):
            line=line.split()
            try:
                resolution[line[0]]=float(line[1])
            except:
                resolution[line[0]]=100

In [11]:
cluster_result['resolution']=cluster_result['pdb'].apply(lambda x: resolution[x])

In [12]:
cluster_result[cluster_result['resolution']<3]

,pdb,affinity,PCV_cluster,main_chain,res_num,pfam,pfam_name,pdb_start,pdb_end,cov_num,...,SCV_cluster,smina,year,Type,heavy_atom_num,ligand_sasa,protein_sasa,complex_sasa,delta_sasa,resolution
0,4i7p,3.50,Lysozyme,A,39.0,PF00959,Phage_lysozyme,24.0,148.0,37.0,...,Cluster 70,3.12597,2013,Refine,6,2.428921,97.99399,97.458916,2.963997,1.60
1,1l83,3.40,Lysozyme,A,39.0,PF00959,Phage_lysozyme,24.0,148.0,37.0,...,Cluster 70,4.88992,1993,Refine,6,2.4389596,86.61278,86.114296,2.937439,1.70
2,1li6,3.80,Lysozyme,A,36.0,PF00959,Phage_lysozyme,24.0,148.0,34.0,...,Cluster 70,3.49193,2002,Refine,6,2.539201,88.206024,87.66072,3.084503,2.00
3,4w52,3.76,Lysozyme,A,38.0,PF00959,Phage_lysozyme,24.0,148.0,36.0,...,Cluster 70,5.08646,2015,Refine,6,2.4466462000000004,90.868965,90.15058,3.165031,1.50
4,188l,3.33,Lysozyme,A,42.0,PF00959,Phage_lysozyme,24.0,148.0,40.0,...,Cluster 70,5.79979,1995,Refine,8,2.9427507,87.55676,86.68145,3.818062,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19434,5ylv,6.82,una_223,A,54.0,PF00689,Cation_ATPase_C,808.0,1018.0,13.0,...,Cluster 688,NaN,2018,General,21,5.5241365,428.88156,426.4585,7.947205,2.80
19435,3ump,2.99,Ribokinase,A,62.0,PF00294,PfkB,1.0,309.0,62.0,...,Cluster 604,NaN,2012,General,31,6.4602895,418.34848,416.47897,8.329803,1.85
19436,4q3r,6.27,Arginase,A,48.0,PF00491,Arginase,20.0,337.0,48.0,...,Cluster 226,NaN,2014,General,28,7.064882300000002,325.52408,323.8590700000001,8.729889,2.17
19437,1ind,10.05,Ig,L,18.0,NaN,NaN,NaN,NaN,NaN,...,Cluster 35,NaN,1993,General,34,7.467189,196.35341,195.50394,8.316666,2.20


## criteria 2: MW < 800 ---> 17435

In [13]:
cluster_result[cluster_result['mol_weight']<800].shape

(17435, 25)

## criteria 3: not contain metal ion

In [42]:
metals=['ZN','MG','MN','HG','CO','NA','CD']

In [44]:
i=0
check_num=[]
no_metal_num=[]
others=[]
had_metal=[]
with open ("/pubhome/hzhu02/models/Redocked_pose/split_dataset/SPA_available/out", 'r') as f:
    for line in f.readlines():
        i+=1
        if i%2==0:
            line=line.split()
            over=list(set(line).intersection(set(metals)))
            if len(line) != 1 and len(over)>0:
                had_metal.append(i)
            elif len(line) !=1 and len(over)==0:
                check_num.append(i)
            elif len(line)==1 and line[0] =="O":
                no_metal_num.append(i)
            else:
                others.append(i)


In [45]:
len(had_metal)

4251

In [46]:
len(check_num)

3162

In [47]:
len(no_metal_num)

11888

In [49]:
len(others)

142

In [51]:
others_check_pdb=[]
others_check_content=[]
had_metal_pdb=[]
i=0
with open ("/pubhome/hzhu02/models/Redocked_pose/split_dataset/SPA_available/out", 'r') as f:
    for line in f.readlines():
        i+=1
        if i+1 in others+check_num:
            others_check_pdb.append(line.strip())
        if i in others+check_num:
            others_check_content.append(line.strip())
        if i+1 in had_metal:
            had_metal_pdb.append(line.strip())

In [52]:
check=pd.DataFrame({'pdb':others_check_pdb, "hetatm":others_check_content})

In [56]:
check=check.sort_values(by=['hetatm'])

In [57]:
check.to_csv("/pubhome/hzhu02/models/Redocked_pose/split_dataset/SPA_available/check_metal.csv", index=False)

In [58]:
other_metal=pd.read_csv("/pubhome/hzhu02/models/Redocked_pose/split_dataset/SPA_available/other_metal", header=None)

In [60]:
all_had_metal_pdb = had_metal_pdb+other_metal[0].to_list()

In [61]:
cluster_result['metal']=cluster_result['pdb'].apply(lambda x: True if x in all_had_metal_pdb else False)

## combine above three criteria ---> 11748

In [62]:
cluster_result[(cluster_result['mol_weight']<800)&(cluster_result['resolution']<3)&(cluster_result['metal']==False)]

,pdb,affinity,PCV_cluster,main_chain,res_num,pfam,pfam_name,pdb_start,pdb_end,cov_num,...,year,Type,heavy_atom_num,ligand_sasa,protein_sasa,complex_sasa,delta_sasa,resolution,pocket_chain_num,metal
0,4i7p,3.50,Lysozyme,A,39.0,PF00959,Phage_lysozyme,24.0,148.0,37.0,...,2013,Refine,6,2.428921,97.99399,97.458916,2.963997,1.60,1,False
1,1l83,3.40,Lysozyme,A,39.0,PF00959,Phage_lysozyme,24.0,148.0,37.0,...,1993,Refine,6,2.4389596,86.61278,86.114296,2.937439,1.70,1,False
2,1li6,3.80,Lysozyme,A,36.0,PF00959,Phage_lysozyme,24.0,148.0,34.0,...,2002,Refine,6,2.539201,88.206024,87.66072,3.084503,2.00,1,False
3,4w52,3.76,Lysozyme,A,38.0,PF00959,Phage_lysozyme,24.0,148.0,36.0,...,2015,Refine,6,2.4466462000000004,90.868965,90.15058,3.165031,1.50,1,False
4,188l,3.33,Lysozyme,A,42.0,PF00959,Phage_lysozyme,24.0,148.0,40.0,...,1995,Refine,8,2.9427507,87.55676,86.68145,3.818062,1.80,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19419,8lpr,6.62,Peptidase_PA,A,46.0,NaN,NaN,NaN,NaN,NaN,...,1993,General,29,6.737478,84.06114000000002,81.84904499999998,8.949577,2.25,1,False
19423,3rv4,2.30,ATP-grasp,A,62.0,PF02786,CPSase_L_D2,115.0,324.0,57.0,...,2011,Refine,27,5.760121,319.7489,318.1394,7.369629,1.98,1,False
19426,2rio,4.69,pkinase,A,54.0,PF00069,Pkinase,17.0,299.0,54.0,...,2008,Refine,27,5.7360744000000015,369.14624,366.65628,8.226044,2.40,1,False
19435,3ump,2.99,Ribokinase,A,62.0,PF00294,PfkB,1.0,309.0,62.0,...,2012,General,31,6.4602895,418.34848,416.47897,8.329803,1.85,3,False


## pocket chain num

In [26]:
pocket_chain=pd.read_csv("/pubhome/hzhu02/models/Redocked_pose/split_dataset/cluster/PCV/step_file/pdbbind_v2020_general_step_1_all_pocket_chain_info.csv")
more_than_one_chain = 0
one_chain = 0
others=0
chain_num={}
for pdb in pdb_list:
    chain=pocket_chain[pocket_chain['pdb']==pdb]
    chain_num[pdb]=chain.shape[0]
    if chain.shape[0] > 1:
        more_than_one_chain+=1
    elif chain.shape[0] ==1:
        one_chain+=1
    else:
        others+=1


In [27]:
more_than_one_chain

4150

In [28]:
one_chain

15289

In [29]:
others

0

In [30]:
cluster_result['pocket_chain_num'] = cluster_result['pdb'].apply(lambda x : chain_num[x])

In [ ]:
cluster_result.to_csv("/pubhome/hzhu02/models/Redocked_pose/split_dataset/SPA_available/pdbbind_spa_filter_results.csv", index=False)